In [ ]:
import os

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [ ]:
meshcat = StartMeshcat()

In [ ]:
def make_stack():
    output = ''
    num_layers = 18
    num_blocks_per_layer = 3
    block_width = 0.2
    block_height = 0.1
    for i in range(num_layers):
        # alternate block orientations
        for j in range(num_blocks_per_layer):
            layer_in_x_direction = i % 2 == 0
            x = (block_width * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (block_width * (j - 1))
            z = block_height * i
            orientation = "{ deg: [0, 0, 90] }" if layer_in_x_direction else "{ deg: [0, 0, 0] }" # TODO: adjust based on sdf
            output += f"""
            - add_model:
                name: block_{i * 3 + j}
                file: file://block.sdf
                default_free_body_pose:
                    base_link_airpod:
                        translation: [{x}, {y}, {z}]
                        rotation: !Rpy {orientation}
            """
    return output

In [ ]:
def create_scene():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: your_model
    file: file://{your_model_filename}
    default_free_body_pose:
        base_link_airpod:
            translation: [0.2, -0.3, 0.46]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
"""

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()